# Retroactive Query Evaluation for Iterative AI/ML
### Epic Retreat, Napa, CA, 2023

FlorDB Demo

repository: github.com/ucbrise/flor

In [ ]:
%pip install flordb

In [ ]:
# Download ML Tutorials
!git clone https://github.com/ucbepic/ml_tutorial.git /content/ml_tutorial

In [ ]:
import os
os.chdir('/content/ml_tutorial')

In [ ]:
!git checkout -b flor.shadow

# Let's have a look at `main.py` →

```python
...
Flor.checkpoints(model, optimizer)
for epoch in Flor.loop(range(num_epochs)):
    for i, (images, labels) in Flor.loop(enumerate(train_loader)):
      ...
```


In [ ]:
!code /content/ml_tutorial/main.py

In [ ]:
!python main.py --flor EpicDemo

In [ ]:
import flor
from flor import log_records, full_pivot
pvt = full_pivot(log_records())
pvt

In [ ]:
from IPython.display import display
from IPython.core.display import Image
from pathlib import PurePath

pvt = full_pivot(log_records())
for projid,runid,tstamp,vid,cf_matrix in pvt[
    list(flor.DATA_PREP) + ['cf_matrix']].drop_duplicates().values:
  if not isinstance(cf_matrix, float):
    p = PurePath(cf_matrix)
    display(Image(p))

# Part 2

hey I have log records from a past run

1. I've already shown you how to make sure flor is checkpointing & committing on every run (low overhead)
2. Examing flor logs over multiple versions to see what's known and not yet known from those runs.
3. Show how to generate new log records from past runs.

In [ ]:
!git branch -r

In [ ]:
!git checkout flor.shadow.compressed

# Let's look at `main.py` →

It's someone else's logging

In [ ]:
!code /content/ml_tutorial/main.py

In [ ]:
import flor
from flor import log_records, full_pivot
pvt = full_pivot(log_records())
pvt

In [ ]:
pvt[list(flor.DATA_PREP) 
+ ['cf_matrix', 'hidden_size', 
   'lr', 'batch_size', 'accuracy']].drop_duplicates()

In [ ]:
flor.replay(['cf_matrix'], 'cf_matrix.isna()', 'main.py')

In [ ]:
from IPython.display import display
from IPython.core.display import Image
from pathlib import PurePath

pvt = full_pivot(log_records())
for projid,runid,tstamp,vid,cf_matrix,accuracy in pvt[
    list(flor.DATA_PREP) + ['cf_matrix', 'accuracy']].drop_duplicates().values:
  if not isinstance(cf_matrix, float):
    p = PurePath(cf_matrix)
    print((runid, str(tstamp), accuracy))
    display(Image(p))